# The Drazin Inverse

### Import libraries

In [129]:
import numpy as np
import pandas as pd
import scipy.linalg as la

## Problem 1

In [18]:
def isDrazin(A, k, AD):
    
    m, n = A.shape
    md, nd = AD.shape
    if not m == n or not md == nd:
        raise ValueError('A and AD are required to be square')
    
    ADA = AD @ A
    cond1 = np.allclose(A @ AD, ADA)
    Ak = np.linalg.matrix_power(A, k)
    Akp1 = A @ Ak
    cond2 = np.allclose(Akp1 @ AD, Ak)
    cond3 = np.allclose(ADA @ AD, AD)
    
    cond = cond1 and cond2 and cond3
    return cond

In [19]:
A = np.eye(4)
A[3,3] = 0
A[[0,1,2],[1,2,3]] = 3
AD = np.array([
    [1,-3,9,81],
    [0,1,-3,-18],
    [0,0,1,3],
    [0,0,0,0]
])
k = 1
isDrazin(A, k, AD)

True

In [20]:
BD = np.zeros((3,3))
B = np.array([
    [1,1,3],
    [5,2,6],
    [-2,-1,-3]
])
k=3
isDrazin(B, k, BD)

True

## Problem 2

In [35]:
def Drazin(A, tol):
    m, n = A.shape
    if not m == n:
        raise ValueError('A is required to be square')
    
    f1 = lambda x: abs(x) > tol
    Q1, S, k1 = la.schur(A, sort = f1)
    f2 = lambda x: abs(x) <= tol
    Q2, T, k2 = la.schur(A, sort = f2)
    
    U = np.column_stack((S[:,:k1], T[:,:n-k1]))
    Uinv = la.inv(U)
    V = Uinv @ A @ U
    Z = np.zeros((n,n))
    if not k1 == 0:
        Minv = la.inv(V[:k1,:k1])
        Z[:k1,:k1] = Minv
    
    return U @ Z @ Uinv

In [39]:
np.allclose(Drazin(A, 1e-2), AD)

True

In [38]:
np.allclose(Drazin(B, 1e-2), BD)

True

## Problem 3

In [160]:
def graph_res(A, tol = 1e-2):
    
    m, n = A.shape
    if not m == n:
        raise ValueError('A is supposed to be square')
    
    D = np.diag(np.sum(A + A * np.eye(n), axis = 1))
    L = D - A
    
    R = np.zeros((n,n))
    for i in range(n):
        for j in range(n):
            if not i == j:
                Ltilde = np.copy(L)
                Ltilde[j,:] = np.eye(n,n)[j,:]
                R[i,j] = Drazin(Ltilde, tol)[i,i]
    
    return R

In [122]:
A = np.zeros((4,4))
rows = [0,1,1,2,2,3]
cols = [1,0,2,1,3,2]
A[rows,cols] = 1

R = graph_res(A)
np.allclose(R[0,3], 3)

True

In [123]:
B = 1 - np.eye(2)
R = graph_res(B)
np.allclose(R[0,1], 1)

True

In [125]:
C = np.ones((3,3)) - np.eye(3)
R = graph_res(C)
np.allclose(R[0,2], 2/3)

True

In [126]:
D = (1 - np.eye(2)) * 3
R = graph_res(D)
np.allclose(R[0,1], 1/3)

True

In [127]:
E = (1 - np.eye(2)) * 2
R = graph_res(E)
np.allclose(R[0,1], 1/2)

True

In [128]:
F = (1 - np.eye(2)) * 4
R = graph_res(F)
np.allclose(R[0,1], 1/4)

True

## Problem 4

In [205]:
class LinkPredictor:
    """  Class to perform link prediction
    """
    def __init__(self, file):
        
        graph = pd.read_csv(file, header = None).as_matrix()
        m, n = graph.shape
        
        nodes, idx = np.unique(graph, return_inverse = True)
        self.nodes = nodes
        
        idx = idx.reshape((m, n))
        self.len = len(nodes)
        Adj = np.zeros((self.len, self.len))
        
        for i in range(m):
            Adj[idx[i, 0], idx[i, 1]] = 1
            Adj[idx[i, 1], idx[i, 0]] = 1
        self.Adj = Adj
        
        R = graph_res(Adj)
        self.R = R
        self.maxR = R.max()
        
    def predict_link(self, node = None):
    
        if node == None:
            Adj_cp = np.copy(self.Adj)
            R_cp = np.copy(self.R)
            newL = R_cp + self.maxR * Adj_cp + self.maxR * np.eye(self.len)
            idx_min = np.unravel_index(newL.argmin(), newL.shape)
            return (self.nodes[idx_min[0]], self.nodes[idx_min[1]])
        
        else: 
            if node in self.nodes:
                idx = np.where(self.nodes == node)[0][0]
                Adj_cp = np.copy(self.Adj)
                R_cp = np.copy(self.R)
                newL = R_cp + Adj_cp + np.eye(self.len)
                newL = newL[:,idx]
                idx_min = np.unravel_index(newL.argmin(), newL.shape)
                return self.nodes[idx_min[0]]
                
            else:
                raise ValueError("The node must be in the network.")
                
        
    def add_link(self, node1, node2):
    
        if node1 in self.nodes and node2 in self.nodes:
            idx1 = np.where(self.nodes == node1)[0][0]
            idx2 = np.where(self.nodes == node2)[0][0]
            
            self.Adj[idx1,idx2] = self.Adj[idx1,idx2] + 1
            self.Adj[idx2,idx1] = self.Adj[idx2,idx1] + 1
            self.R = graph_res(self.Adj)
        else:
            raise ValueError("node1 and node2 must be present in the network.")
        

In [206]:
social_ntw = LinkPredictor('social_network.csv')
social_ntw.predict_link()

('Oliver', 'Emily')

In [207]:
social_ntw.predict_link('Melanie')

'Carol'

In [208]:
social_ntw.predict_link('Alan')

'Sonia'

In [209]:
social_ntw.add_link('Alan', 'Sonia')
social_ntw.predict_link('Alan')

'Piers'

In [210]:
social_ntw.add_link('Alan', 'Piers')
social_ntw.predict_link('Alan')

'Abigail'